## Bootstrap confidence intervals

In this notebook we compute the different bootstrap confidence intervals
discussed in class (Bootstrap-t, percentile, hybrid and bias-corrected 
accelerated (BCa) percentile). Because we did not discuss in
class how to 
estimate the "acceleration" coefficient for the BCa approach, 
the latter CI be computed using the
function `bcanon` of package `bootstrap`.

We consider two examples. In the first one 
a simple the interest is in constructing a confidence 
interval for the average nightly Airbnb prices 
in Vancouver. The data includes listings in Vancouver 
for September 2020.
More details can be found in Section 10.4 of 
(Data Science, A First Introduction, 2022, Timbers, Campbell
and Lee)[https://datasciencebook.ca/]. These data are
(naturally) highly skewed. 

The second example is concerned with constructing 
a confidence interval for the ratio of two means. 
The data contains historical population levels for 
49 cities in the US for the years 1920 and 1930,
and the interest in in the ratio of the 
mean population for each of the two years.
The data is analyzed in more detail in 
`Davison, AC., Hinkley, DV. (1997). Bootstrap methods and their applications`
(link available on Canvas). 


### Airbnb  example

<!-- 10.4 of https://datasciencebook.ca/ -->
The data is available in the `data/listings.csv` file. 
There are different ways to load these data in `R`, we use
the simplest one using the "base" function `read.table` that
returns a simple `data.frame`, from which we can 
extract the `price` variable:

In [ ]:
airbnb <- read.table("data/listings.csv", header=TRUE, sep=',')
x <- airbnb$price

Note that these data are highly skewed

In [ ]:
hist(x)

For illustration purposes we consider a very small
subset of size `n = 25` from the 4594 available 
listings.

In [ ]:
n <- 25
set.seed(17)
x0 <- sample(x, n)

You can check that this subsample is also highly skewed. 
We will use bootstrap to construct a confidence interval
for the average listing price for September 2020. We
might consider the average of the full data as the 
"true" parameter (our "target"). 

#### CLT-based CI

Using a simple Normal approximation to the distribution 
of the sample mean, we obtain the following 
95% confidence interval:

In [ ]:
alpha <- 0.05
n <- length(x0)
lo0 <- mean(x0) - qnorm(.975) * sd(x0) / sqrt(n)
up0 <- mean(x0) + qnorm(.975) * sd(x0) / sqrt(n)
c(lo0, up0)

Note that the "true" mean is `r mean(x)`. 

In many applications it is recommended to work with 
the logarithm of the data to reduce skeweness (specially
when dealing with prices). You can check that indeed the
log of the data has a much more symmetric distribution. 
Thus we can construct a CLT-based CI for the mean of the
logarithm of the prices.

In [ ]:
tmp0 <- mean(log(x0)) - qnorm(.975) * sd(log(x0)) / sqrt(n)
tmp1 <- mean(log(x0)) + qnorm(.975) * sd(log(x0)) / sqrt(n)

However, how to back-transform this to a CI for the mean
of the original data can be a bit delicate. 
We get

In [ ]:
si2 <- var(x0)
mu <- mean(x0)
lo1 <- exp(tmp0 + si2/2/mu^2)
up1 <- exp(tmp1 + si2/2/mu^2)
c(lo1, up1)

#### Percentile bootstrap CI

We use 10000 bootstrap samples to estimate the bootstrap
sampling distribution of the sample mean. The CI is given
by the corresponding percentiles.

In [ ]:
M <- 1e6
p.boot <- vector('numeric', M)
set.seed(123)
for(j in 1:M) {
  p.boot[j] <- mean(x0[sample(n, repl=TRUE)])
}
lo2 <- quantile(p.boot, alpha/2)
up2 <- quantile(p.boot, 1-alpha/2)
c(lo2, up2)

#### Hybrid CI

Here we assume that the sampling distribution of the 
"centered" statistic 
(hat(theta) - theta) may not depend 
("much") on unknowns (i.e. that is an approximate pivot). 
We use bootstrap to estimate
its sampling distribution, and the corresponding 
percentiles as estimators of the percentiles of the
sampling distribution of the statistic of interest.

In [ ]:
theta0 <- mean(x0)
m.boot <- p.boot - theta0
up3 <- theta0 - quantile(m.boot, alpha/2)
lo3 <- theta0 - quantile(m.boot, 1-alpha/2)
c(lo3, up3)

#### Bootstrap-T

In this approach we bootstrap a studentized 
statistic, which we expect to being even closer
to a pivot. To do this efficiently we need
an expression for the variance of the statistic
of interest that can be re-computed for each
bootstrap sample. Since we're bootstrapping the
sample mean, we now that its estimated variance is 
the variance of the sample being averaged
(in this case: the sample variance of the 
bootstrap samples)
divided by the sample size n.

In [ ]:
M <- 1e6
m.boot <- vector('numeric', M)
set.seed(123)
for(j in 1:M) {
  boot.sam <- x0[sample(n, repl=TRUE)]
  m.boot[j] <- (mean(boot.sam) - theta0)/sqrt(var(boot.sam)/n)
}
up4 <- theta0 - sqrt(var(x0)/n) * quantile(m.boot, alpha/2)
lo4 <- theta0 - sqrt(var(x0)/n) * quantile(m.boot, 1 - alpha/2)
c(lo4, up4)

#### Bias corrected

We use the same estimate of the sampling distribution
that we constructed for the "simple" percentile
approach. The recomputed statistics were stored in
the vector `p.boot`

In [ ]:
(z0hat <- qnorm( mean( p.boot <= mean(x0) ) ) )
( alpha.tilde <- pnorm( 2*z0hat + qnorm(.025) ) )
lo5 <- quantile(p.boot, alpha.tilde/2)
up5 <- quantile(p.boot, 1-alpha.tilde/2)
c(lo5, up5)

#### Bias corrected accelerated

In [ ]:
set.seed(123)
tmp <- bootstrap::bcanon(x=x0, nboot=1e6, theta=mean, alpha=c(0.025, 0.975))
tmp$confpoints

As a  small sanity check, we can verify that the
estimated bias correction in this implementation 
"matches" what we computed above:

In [ ]:
c(z0hat, tmp$z0)

## Bivariate

We now consider a less simple example where 
the interest is in constructing 
a confidence interval for the ratio of two means. 
The data are pairs of population levels for 
49 cities in the US for the years 1920 and 1930. 
The data are available in the package `boot`, as
a 2-column matrix:

In [ ]:
ci <- boot::bigcity

The point estimator is the ratio of the two means:

In [ ]:
(theta0 <- mean(ci[, 1])/mean(ci[, 2]))

We now use the bootstrap to construct CI's for 
the popuulation value of this ratio. 

The calculations are very similar to the ones 
before. Note that in this case we cannot really use
the Bootstrap-t method, since there is no simple 
(consistent) estimator of the variance of the ratio of two
sample means. 

#### Percentile

For the simple percentile bootstrap confidence interval, 
we estimate the bootstrap cdf estimator using the computer, 
as before. The bootstrap sample is a random sample 
(with replacement) drawn from the 
49 pairs of population numbers, and for each of these
bootstrap samples we compute the ratio of the means of
each column:

In [ ]:
n <- dim(ci)[1]
M <- 1e6
p.boot <- vector('numeric', M)
set.seed(123)
for(j in 1:M) {
  tmp <- sample(n, repl=TRUE)
  tmp2 <- colMeans(ci[tmp, ])
  p.boot[j] <- tmp2[1] / tmp2[2] # mean(ci[tmp, 1])/mean(ci[tmp,2])
}
lo1 <- quantile(p.boot, alpha/2)
up1 <- quantile(p.boot, 1-alpha/2)
c(lo1, up1)

#### Hybrid

For the hybrid approach we center our statistic 
with the point estimator found with the whole data set
(denoted `theta0` in the code below):

In [ ]:
m.boot <- p.boot - theta0
up2 <- theta0 - quantile(m.boot, alpha/2)
lo2 <- theta0 - quantile(m.boot, 1-alpha/2)
c(lo2, up2)

#### BC

As before, for the bias corrected approach we can 
re-use the bootstrap cdf estimator obtained for the
simple percentile approach (its values were stored
in the `p.boot` object, which we use again here, 
but use the modified percentiles):

In [ ]:
(z0hat <- qnorm( mean( p.boot <= theta0 ) ) )
( alpha.tilde <- pnorm( 2*z0hat + qnorm(.025) ) )
lo5 <- quantile(p.boot, alpha.tilde/2)
up5 <- quantile(p.boot, 1-alpha.tilde/2)
c(lo5, up5)

#### BCa

In this case to use the implementation in 
`bootstrap::bcanon` we need to
modify our code. This function is written 
for bootstrapping univariate statistics. A simple
trick we can use is to write our estimator as 
a function of the bootstrapped indices
(and the data), and then use `bootstrap::bcanon` to
bootstrap from the "data" 1, 2, ..., n. 
Our "estimator" is

In [ ]:
theta <- function(ind, data) {
    tmp <- colMeans(data[ind,])
    return(tmp[1] / tmp[2]) # mean(data[ind, 1])  / mean(data[ind, 2])
    }

And then

In [ ]:
M <- 1e6
n <- dim(ci)[1]
set.seed(123)
tmp <- bootstrap::bcanon(x=1:n, nboot=M, theta=theta, data=ci, alpha=c(0.025, .975))
tmp$confpoints